<a href="https://colab.research.google.com/github/MubshAli/StockEval/blob/main/intrinsicValue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yahoo_fin --upgrade

     |████████████████████████████████| 81 kB 6.6 MB/s 
     |████████████████████████████████| 83 kB 1.2 MB/s 
     |████████████████████████████████| 127 kB 44.0 MB/s 
     |████████████████████████████████| 103 kB 50.1 MB/s 
  Created wheel for fake-useragent: filename=fake_useragent-0.1.11-py3-none-any.whl size=13502 sha256=dc609e917f3b2aba8ac1fd75af19cff2b179cd9b5b319ec7a6862c647870ffeb
  Stored in directory: /root/.cache/pip/wheels/ed/f7/62/50ab6c9a0b5567267ab76a9daa9d06315704209b2c5d032031
  Created wheel for parse: filename=parse-1.19.0-py3-none-any.whl size=24591 sha256=ff852c58660c4b9a8248f0c35cbc382fd901da6641ccb1348c292365898a2ce1
  Stored in directory: /root/.cache/pip/wheels/9c/aa/cc/f2228050ccb40f22144b073f15a2c84f11204f29fc0dce028e
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6065 sha256=f8f1aa259f5c6eba3f88e606134c4215e9016d12324d43269aecb5b81618ed44
  Stored in directory: /root/.cache/pip/wheels/73/ad/a4/0dff4a6ef231fc0dfa12ffbac2a36ce

In [2]:
# innstall java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
 
# install spark (change the version number if needed)
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
 
# unzip the spark file to the current folder
!tar xf spark-3.0.0-bin-hadoop3.2.tgz
 
# set your spark folder to your system path environment. 
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"
 
 
# install findspark using pip
!pip install -q findspark

In [3]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [4]:
import findspark
findspark.init()

from pyspark import SparkContext, SparkConf

spark_conf = SparkConf()\
  .setAppName("YourTest")\
  .setMaster("local[*]")

sc = SparkContext.getOrCreate(spark_conf)

In [5]:
rdd = sc.parallelize([1,2,3])
rdd.take(1)

[1]

In [6]:
pip install requests_html

In [7]:
from yahoo_fin.stock_info import get_data
#get_analysts_info()

In [8]:
import yahoo_fin.stock_info as si

{'1y Target Est': 167.13,
 '52 Week Range': '103.10 - 157.26',
 'Ask': '148.72 x 1100',
 'Avg. Volume': 78099357.0,
 'Beta (5Y Monthly)': 1.2,
 'Bid': '148.71 x 2200',
 "Day's Range": '148.70 - 155.48',
 'EPS (TTM)': 5.11,
 'Earnings Date': 'Oct 27, 2021 - Nov 01, 2021',
 'Ex-Dividend Date': 'Aug 06, 2021',
 'Forward Dividend & Yield': '0.88 (0.59%)',
 'Market Cap': '2.463T',
 'Open': 155.0,
 'PE Ratio (TTM)': 29.16,
 'Previous Close': 154.07,
 'Quote Price': 148.97000122070312,
 'Volume': 140893235.0}

In [9]:
from pandas import *

In [10]:
pip install get-all-tickers

  Created wheel for get-all-tickers: filename=get_all_tickers-1.8-py3-none-any.whl size=4247 sha256=1cd8896d598df0ef325d337532e9d17712d106392572727e5e5523eec04f1129
  Stored in directory: /root/.cache/pip/wheels/f0/8d/8f/25b9632788de8771fb8be22529bbe0ba5ae5446f3be65b1ab3
Successfully built get-all-tickers


In [11]:
from get_all_tickers import get_tickers as gt

In [12]:
s = si.tickers_dow()
m = si.tickers_nasdaq()
a = si.tickers_sp500()
o = si.tickers_other()
b = s + a + m + o
len(b)
rdd_companies = sc.parallelize(b)

In [24]:
from pyspark.sql import SparkSession
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
schema = StructType([
    StructField('Category', StringType(), True),
    StructField('Count', IntegerType(), True),
    StructField('Description', StringType(), True)
])

df = spark.createDataFrame(rdd_companies,schema)


In [25]:
df.show()

Py4JJavaError: ignored

In [13]:
lst = []
for i in range(0,10000000):
  lst.append(i)

In [32]:
lst_rdd = sc.parallelize(lst)
lst_rdd.take(5)
def square(x):
  return x ** 2
squared = lst_rdd.map(square)
y = squared.take(1000)
type(b[0])

str

<class 'str'>


In [33]:
result = []
rdd_companies = sc.parallelize(b)
def intrinsicVal(x):
  quote_table = si.get_quote_table(x,dict_result=True)
  anal = si.get_analysts_info(x)
  p = anal['Growth Estimates']
  val = p[x].values[4]
  s = ""
  for i in range(len(val)):
    if val[i] == "%" or val[i] == ".":
      s = s
    else:
      s += val[i]
  s = int(s)
  div = 10000
  s = s/div
  growth_rate = s
  min_ror = 0.15
  margin_of_safety = 0.5
  p_e_ratio = quote_table["PE Ratio (TTM)"]
  eps = quote_table["EPS (TTM)"]
  curr_price = quote_table["Quote Price"]
  eps_after_ten = eps
  for i in range(9):
    eps_after_ten += growth_rate*eps_after_ten
  share_price_after_ten = eps_after_ten * p_e_ratio
  expected_price = share_price_after_ten
  for i in range(9):
    expected_price = expected_price/(1+min_ror)
  curr_value = expected_price
  #print(curr_value)
  #add_to_result = (x, curr_value)
  #print(add_to_result)
  #result.append(add_to_result)
  return curr_value

companies_rdd = rdd_companies.map(intrinsicVal)
companies_rdd.take(20)
#companies_rdd.take(5)

#companies_rdd.take(11600)


# for i in range(20):
#   company = b[i]
#   #print(company)
#   quote_table = si.get_quote_table(company, dict_result=True)
#   anal= si.get_analysts_info(company)
#   x = anal['Growth Estimates']
#   val = x[company].values[4]
#   print(val)
#   s = ""  
#   for i in range(len(val)):
#     if val[i] == "%" or val[i] == ".":
#       s = s
#     else:
#       s += val[i]
#   #print(s)
#   l = len(s)
#   div = 10000
#   #print(div)
#   s = int(s)
#   s = s/div
#   print(s)

#   growth_rate = s
#   min_ror = 0.15
#   margin_of_safety = 0.50
#   p_e_ratio = quote_table["PE Ratio (TTM)"]
#   eps = quote_table["EPS (TTM)"]
#   curr_price = quote_table["Quote Price"]
#   eps_after_ten = eps
#   for i in range(9):
#     eps_after_ten += growth_rate*eps_after_ten
#   share_price_after_ten = eps_after_ten * p_e_ratio
#   expected_price = share_price_after_ten
#   for i in range(9):
#     expected_price = expected_price/(1+min_ror)
#   curr_value = expected_price
#   #print(curr_value)
#   add_to_result = (company, curr_value)
#   print(add_to_result)
#   result.append(add_to_result)

# print(result)



KeyboardInterrupt: ignored

In [ ]:
import math
print(q)


growth_rate = 0.15
min_ror = 0.15
margin_of_safety = 0.50
p_e_ratio = q["PE Ratio (TTM)"]

eps = q["EPS (TTM)"]
x = float('nan')
if(math.isnan(p_e_ratio)):
  print("GG")
curr_price = q["Quote Price"]
eps_after_ten = eps
for i in range(9):
  eps_after_ten += growth_rate*eps_after_ten
share_price_after_ten = eps_after_ten * p_e_ratio
expected_price = share_price_after_ten
for i in range(9):
  expected_price = expected_price/(1+min_ror)
curr_value = expected_price



#print(curr_value)
# add_to_result = (company, curr_value)
# print(add_to_result)
# result.append(add_to_result)

{'1y Target Est': 268.22, '52 Week Range': '141.58 - 278.57', 'Ask': '211.38 x 900', 'Avg. Volume': 10777435.0, 'Beta (5Y Monthly)': 1.62, 'Bid': '211.30 x 1200', "Day's Range": '210.13 - 215.89', 'EPS (TTM)': -20.55, 'Earnings Date': 'Oct 26, 2021 - Nov 01, 2021', 'Ex-Dividend Date': 'Feb 13, 2020', 'Forward Dividend & Yield': 'N/A (N/A)', 'Market Cap': '122.986B', 'Open': 215.16, 'PE Ratio (TTM)': nan, 'Previous Close': 213.94, 'Quote Price': 210.3000030517578, 'Volume': 6959706.0}
GG


False

In [ ]:
print(quote_table)

{'1y Target Est': 67.29, '52 Week Range': '44.33 - 71.38', 'Ask': '60.68 x 800', 'Avg. Volume': 4141314.0, 'Beta (5Y Monthly)': 1.72, 'Bid': '60.67 x 1400', "Day's Range": '60.34 - 61.44', 'EPS (TTM)': 2.65, 'Earnings Date': 'Oct 20, 2021 - Oct 25, 2021', 'Ex-Dividend Date': 'Aug 30, 2021', 'Forward Dividend & Yield': '2.80 (4.38%)', 'Market Cap': '45.126B', 'Open': 60.85, 'PE Ratio (TTM)': 22.82, 'Previous Close': 60.28, 'Quote Price': 60.40999984741211, 'Volume': 4093257.0}


In [ ]:
b[0:10]

['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW']